In [162]:
#Import Various Libraries, including Tweepy, a Python library for the Twitter API.
import os
import requests
import pandas as pd
from dotenv import load_dotenv
#%matplotlib inline
import tweepy

load_dotenv()

True

In [163]:
#Pass API Keys to Twitter API and build Tweepy API handler object.
consumer_key = os.getenv('ALT_TWITTER_API_KEY')
consumer_secret = os.getenv('ALT_TWITTER_SECRET_KEY')
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

# Defining the results DataFrame and Search Targets

In [106]:
#Creating a DataFrame for tweet search results data structure.
dict_tweet_structure = {"twitter_user":"","category":[],"time":"","sentiment":"","text":"","tweet_id":"",
                        "tweet_source":"","quote_count":0,"reply_count":0,"retweet_count":0,"fav_count":0}
df_tweets_found = pd.DataFrame(dict_tweet_structure)
df_tweets_found

,twitter_user,category,time,sentiment,text,tweet_id,tweet_source,quote_count,reply_count,retweet_count,fav_count


In [152]:
#Define Search Term Library
dict_search_terms = {'bitcoin':['bitcoin','btc','#bitcoin','#btc',],
                     'generic':['cryptocurrency','blockchain'],
                     'litecoin':['litecoin','ltc','#litecoin', '#ltc'],
                     'ethereum':['ethereum','eth','#ethereum','#eth']   }
#Define Twitter User List
list_twitterers = ['joerogan','elonmusk','officialmcafee','vitalikbuterin']
for person in list_twitterers:
    print(len(person))

8
8
14
14


# Functions

In [6]:
#Function for creating search query string for tweepy.Cursor(api.search). Max query length is 128 for sandbox env.
#In it's current form it supports a bunch of keywords joined by OR, grouped by (), AND from a single tweeter.
def query_creator(list_keywords, twitter_user = "!", mood = "!"):
    query = "("
    for keyword in (list_keywords):
        query += keyword
        if keyword != list_keywords[-1]:
            query += " OR "
        else:
            query += ")"
    if twitter_user != "!":
        query += " from:" + twitter_user
    return query

In [157]:
#This is the core function for tweet searching.
def tweet_search_full(string_query, date_from = "200603210000", date_to = "202101170000", number_tweets = 100):
    for tweets in tweepy.Cursor(api.search_full_archive, query=string_query, environment_name='CryptoSentimentQueryProd',
                                fromDate=date_from, toDate=date_to).items():
        #Primary environment name = CryptoSentimentFullArc
        screen_name = tweets.user.screen_name
        contents = ""
        #if tweets.truncated == True:
        #    contents = tweets.full_text
        #else:
        contents = tweets.text
        category = category_key(contents,dict_search_terms)
        yield [screen_name, category, str(tweets.created_at), "", contents, tweets.id, tweets.source, tweets.quote_count,
               tweets.reply_count, tweets.retweet_count, tweets.favorite_count]

In [164]:
#Verifying DataFrame structure.
df_tweets_found

,twitter_user,category,time,sentiment,text,tweet_id,tweet_source,quote_count,reply_count,retweet_count,fav_count
0,elonmusk,bitcoin,2020-12-20 09:24:37+00:00,,Bitcoin is almost as bs as fiat money,1340588909974200321,Twitter for iPhone,3117,8427,10418,142292
1,elonmusk,bitcoin,2020-12-20 08:21:25+00:00,,Bitcoin is my safe word,1340573003579617280,Twitter Web App,3191,6871,20548,243138
2,elonmusk,,2020-11-17 06:54:03+00:00,,@trylolli @Maisie_Williams 👻 💵 \nhttps://t.co/...,1328592219956252673,Twitter for iPhone,31,85,57,969
3,elonmusk,bitcoin,2020-11-16 22:02:51+00:00,,@Maisie_Williams 🎶 Toss a bitcoin to ur Witcher 🎶,1328458535340949505,Twitter for iPhone,765,1138,3433,66880
4,elonmusk,bitcoin,2020-12-20 09:24:37+00:00,,Bitcoin is almost as bs as fiat money,1340588909974200321,Twitter for iPhone,3117,8426,10418,142292
5,elonmusk,bitcoin,2020-12-20 08:21:25+00:00,,Bitcoin is my safe word,1340573003579617280,Twitter Web App,3191,6871,20549,243138
6,elonmusk,,2020-11-17 06:54:03+00:00,,@trylolli @Maisie_Williams 👻 💵 \nhttps://t.co/...,1328592219956252673,Twitter for iPhone,31,85,57,969
7,elonmusk,bitcoin,2020-11-16 22:02:51+00:00,,@Maisie_Williams 🎶 Toss a bitcoin to ur Witcher 🎶,1328458535340949505,Twitter for iPhone,765,1138,3433,66879
8,elonmusk,bitcoin,2020-12-20 09:24:37+00:00,,Bitcoin is almost as bs as fiat money,1340588909974200321,Twitter for iPhone,3117,8426,10418,142292
9,elonmusk,bitcoin,2020-12-20 08:21:25+00:00,,Bitcoin is my safe word,1340573003579617280,Twitter Web App,3191,6871,20549,243140


In [131]:
#This function searches the last month, but uses the same arguments and formatting as full archive; useful for testing without using up queries.
def tweet_search_month(string_query, date_from = "200603210000", date_to = "202101170000", number_tweets = 100):
    for tweets in tweepy.Cursor(api.search_30_day, query=string_query, environment_name='CryptoSentimentQueryTest',
                                fromDate=date_from, toDate=date_to,).items():
        screen_name = tweets.user.screen_name
        contents = ""
        #if tweets.truncated == True:
        #    contents = tweets.full_text
        #else:
        contents = tweets.text
        
        category = category_key(contents,dict_search_terms)
        yield [screen_name, category, str(tweets.created_at), str(tweets.coordinates), contents, tweets.id, tweets.source, tweets.quote_count,
               tweets.reply_count, tweets.retweet_count, tweets.favorite_count]

In [9]:
#Function for generating a list of categories from the presence of keywords in text.
def category_key(text_block,dict_keywords):
    category_list = []
    output = " "
    contents = text_block.lower()
    for key in dict_keywords:
        for keyword in dict_keywords[key]:
            if keyword in contents:
                category_list.append(key)
    #return output.join(category_list)
    return category_list

# Function usage and testing the DataFrame

In [165]:
#Testing Query Creator
#dict_search_terms = {'bitcoin':['bitcoin','btc','#bitcoin','#btc',],
 #                    'generic':['cryptocurrency','crypto','blockchain', 'struggle']}
query = query_creator(dict_search_terms['generic']+dict_search_terms['bitcoin']+dict_search_terms['ethereum'], 'vitalikbuterin')
#query += " -is:retweet"
date_since_pro = "202001140000"
date_from_pro = "202101170000"
print(query)
print(len(query))
#Query length is limited to 128 characters, max tweets per query is limited to 100.

(cryptocurrency OR blockchain OR bitcoin OR btc OR #bitcoin OR #btc OR ethereum OR eth OR #ethereum OR #eth) from:vitalikbuterin
128


(litecoin OR ltc OR #litecoin OR #ltc OR ethereum OR eth OR #ethereum OR #eth) from:elonmusk
92


In [166]:
#Testing primary search and DataFrame append.
search_results = tweet_search_full(query)

for tweets in search_results:
    print (tweets)
    series_result = pd.Series(tweets, index=df_tweets_found.columns)
    df_tweets_found = df_tweets_found.append(series_result, ignore_index=True)

['VitalikButerin', 'ethereum ethereum', '2020-12-28 22:49:03+00:00', '', 'A much more detailed explanation of my data-availability-based proposal for adding sharding to the ethereum PoS cha… https://t.co/HQ9kpuNO9M', 1343690455435730945, 'Twitter Web App', 13, 120, 168, 873]
['VitalikButerin', '', '2020-12-28 07:08:20+00:00', '', '@woowoocryptoguy Maybe if we set up a scheme where anyone can visit any country if they present proof of liability… https://t.co/1icHZoq85Z', 1343453713940115456, 'Twitter Web App', 72, 49, 162, 531]
['VitalikButerin', 'ethereum', '2020-12-28 07:03:58+00:00', '', 'RT @SBF_Alameda: 1) I talked with @VitalikButerin about L1s: ETH, SOL, others.\n\nHere are some notes.', 1343452615737102336, 'Twitter Web App', 0, 0, 0, 0]
['VitalikButerin', 'generic', '2020-12-28 07:02:20+00:00', '', "@SBF_Alameda Another crucial point is that even if only ~20% of a blockchain's stakers are highly distributed indiv… https://t.co/CD40bP1ZgZ", 1343452206670831616, 'Twitter Web App'

In [167]:
df_tweets_found

,twitter_user,category,time,sentiment,text,tweet_id,tweet_source,quote_count,reply_count,retweet_count,fav_count
0,elonmusk,bitcoin,2020-12-20 09:24:37+00:00,,Bitcoin is almost as bs as fiat money,1340588909974200321,Twitter for iPhone,3117,8427,10418,142292
1,elonmusk,bitcoin,2020-12-20 08:21:25+00:00,,Bitcoin is my safe word,1340573003579617280,Twitter Web App,3191,6871,20548,243138
2,elonmusk,,2020-11-17 06:54:03+00:00,,@trylolli @Maisie_Williams 👻 💵 \nhttps://t.co/...,1328592219956252673,Twitter for iPhone,31,85,57,969
3,elonmusk,bitcoin,2020-11-16 22:02:51+00:00,,@Maisie_Williams 🎶 Toss a bitcoin to ur Witcher 🎶,1328458535340949505,Twitter for iPhone,765,1138,3433,66880
4,elonmusk,bitcoin,2020-12-20 09:24:37+00:00,,Bitcoin is almost as bs as fiat money,1340588909974200321,Twitter for iPhone,3117,8426,10418,142292
5,elonmusk,bitcoin,2020-12-20 08:21:25+00:00,,Bitcoin is my safe word,1340573003579617280,Twitter Web App,3191,6871,20549,243138
6,elonmusk,,2020-11-17 06:54:03+00:00,,@trylolli @Maisie_Williams 👻 💵 \nhttps://t.co/...,1328592219956252673,Twitter for iPhone,31,85,57,969
7,elonmusk,bitcoin,2020-11-16 22:02:51+00:00,,@Maisie_Williams 🎶 Toss a bitcoin to ur Witcher 🎶,1328458535340949505,Twitter for iPhone,765,1138,3433,66879
8,elonmusk,bitcoin,2020-12-20 09:24:37+00:00,,Bitcoin is almost as bs as fiat money,1340588909974200321,Twitter for iPhone,3117,8426,10418,142292
9,elonmusk,bitcoin,2020-12-20 08:21:25+00:00,,Bitcoin is my safe word,1340573003579617280,Twitter Web App,3191,6871,20549,243140


In [94]:
df_tweets_found.loc[0]['text']


'Self imposed struggle is essential for a clear mind. @onnit https://t.co/mvd8yseLc6'

In [168]:
df_tweets_found.to_csv('../data/raw_data/raw_tweets_02.csv',index=False)

# Old, Defunct, Or Reference Code Beyond This Point

In [31]:
#This is the user_timeline method, which seems to return around 500~1000 tweets tops.
ticker = 0
for tweet in tweepy.Cursor(api.user_timeline, id='elonmusk',trim_user=True, max_id=2237531699681981416, count =10000, exclude_replies = True).items():
    #if 'bitcoin' in tweet.text:
    ticker += 1
    print(tweet.text + " " + str(tweet.created_at)+" "+str(tweet.id)+"\n")
print(ticker)

Battery cell production is the fundamental rate-limiter slowing down a sustainable energy future. Very important pr… https://t.co/9ybABEzxCV 2021-01-18 05:52:28+00:00 1351044768030142464

Monty Python is amazing
https://t.co/UJq94IWT88 2021-01-15 06:44:22+00:00 1349970666477527042

RT @SpaceX: Splashdown of Dragon confirmed, completing SpaceX’s 21st @Space_Station resupply mission and the first return of a cargo resupp… 2021-01-14 03:51:19+00:00 1349564729417756672

https://t.co/ho7yGXAS3a 2021-01-13 21:20:05+00:00 1349466269121179653

Today at SpaceX is about practicing Starship engine starts. Ship is held down by massive pins while engines are fir… https://t.co/QtIiWLxXPl 2021-01-13 19:06:00+00:00 1349432529657294848

Legalize comedy 2021-01-13 09:25:42+00:00 1349286488618491904

RT @SpaceX: Separation confirmed! Dragon performing 3 departure burns to move away from the @Space_Station https://t.co/G27ohWvnXj 2021-01-13 01:25:16+00:00 1349165584995512320

Hey you …
Yeah you Queen …
Yo

In [35]:
for tweet in tweepy.Cursor(api.user_timeline, id="elonmusk",tweet_mode="extended").items(100):
    #if 'Python' in tweet.full_text:
        print(tweet.full_text)
        #print(tweet.user['id_str'])

In [ ]:
#Defunct Old Search Method
def tweet_gather(keywords, user_id):
    #contents = ""
    for tweet in tweepy.Cursor(api.user_timeline, id=user_id).items():
        """if tweet.truncated == True:
            contents = tweet.full_text
        else:
            contents = str(tweet.text).lower()"""
        
        contents = tweet.text.lower()
        #contents = contents.lower() 
        for word in keywords:           
            if word.lower() in contents:
                #print(tweet.text + " " + str(tweet.created_at)+"\n")
                """Here we can append to a pd.DataFrame that collects the twitter handle, keywords used, keyword category
                   for ex:'bitcoin' for hits on 'bitcoin' or 'btc', as well as any sentiment rating we create.
                   Ulitmately we can then export that DataFrame to csv file."""
                yield tweet.text, str(tweet.created_at)

In [111]:
#This is the basic api.search; it only returns tweets from the last 7 days.
for tweet in tweepy.Cursor(api.search, q='(cryptocurrency OR crypto OR blockchain OR bitcoin OR struggle OR python)').items(1000):
    print(tweet.text + " " + str(tweet.created_at)+" "+str(tweet.place)+"\n")

@btchudson I’m curious what is your take on the @TheFCA statement on bitcoin &amp; U.K. media publishing it whilst they… https://t.co/dMadFwYRHk 2021-01-20 04:24:59+00:00 None

RT @khadijahlatfy: Stress betul la aku buat assignment kat rumah. Lecturer please understand the struggle😩 https://t.co/IXHoY5EI5E 2021-01-20 04:24:58+00:00 None

@shawarmacommie Thank you, comrade 😁 it's good to have you back on the timeliness too! You always make such good ef… https://t.co/iF78GJLZvr 2021-01-20 04:24:58+00:00 None

RT @feelinfroggy181: $20 Giveaway 💜💜

💫 Follow @feelinfroggy181 @inez_crypto
💫 Retweet

Ends in the morning. 💜

id:  dealbase0001 2021-01-20 04:24:58+00:00 None

@BigNiceGiveaway @inez_crypto 123 2021-01-20 04:24:57+00:00 None

@inez_crypto Thank you 2021-01-20 04:24:57+00:00 None

RT @CRYPTOFIED1: I’ll give someone $30 crypto or cash in 12 hours that retweets this joins discord and tweets this text with pic

$BTC is g… 2021-01-20 04:24:57+00:00 None

RT @PeanutTrade: Our first ☝️ a

In [ ]:
#Function for creating search query string for tweepy.Cursor(api.search_full_archive).
def query_creator_full(list_keywords, twitter_user = "!", mood = "!"):
    query = " "
    query = query.join(list_keywords)
    #print(query)
    if twitter_user != "!":
        query += " from:" + twitter_user
        #print(query)
    return query